Загрузка модулей

In [10]:
!pip install python-Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 51.8 MB/s eta 0:00:00


Загрузка библиотек

In [11]:
import pandas as pd
import numpy as np
import re
import Levenshtein as lev
import sys

# **Загрузка исходных данных**

In [3]:
from google.colab import files

uploaded = files.upload()

Saving reviews.xlsx to reviews.xlsx


In [4]:
df=pd.read_excel('/content/reviews.xlsx')

# **Вспомогательные функции**

In [12]:
# --- Функция вывода статистики классов ---
def class_statistics(df):
    """
    Возвращает таблицу статистики по уникальным спискам из колонки 'Класс'.

    Таблица содержит:
        - Уникальный класс (в виде строки)
        - Количество таких классов
        - Процент от общего числа строк
    """
    if 'Класс' not in df.columns:
        raise ValueError("В датафрейме отсутствует колонка 'Класс'")

    # Нормализуем списки к кортежам для подсчёта
    normalized = df['Класс'].apply(tuple)

    # Подсчёт частоты
    stats = normalized.value_counts().reset_index()
    stats.columns = ['Класс', 'Количество']

    # Преобразуем кортежи в читаемые строки (например, '(0,)' → '[0]')
    stats['Класс'] = stats['Класс'].apply(lambda x: str(list(x)))

    # Процент
    total = len(df)
    stats['Процент'] = (stats['Количество'] / total * 100).round(2)

    # Сортировка по убыванию
    stats = stats.sort_values(by='Количество', ascending=False)

    # Добавляем строку "Всего"
    stats.loc[len(stats)] = ['Всего', total, 100.00]

    return stats.reset_index(drop=True)

In [13]:
# --- Функция присвоения класса ---
def set_class(df, idx, code):
    """
    Присваивает класс с указанным кодом.
    Если [0] → заменить на [code]
    Если нет 0 и нет code → добавляем code
    Иначе → ничего не делаем
    """
    current_class = df.at[idx, 'Класс']

    if current_class == [0]:
        df.at[idx, 'Класс'] = [code]
    elif 0 not in current_class and code not in current_class:
        df.at[idx, 'Класс'] = current_class + [code]

    return df

In [14]:
# --- Функция очищает текстовые колонки ---
def clean_text_columns(df, columns=['Отзыв', 'Товар+Отзыв']):
    """
    Очищает указанные текстовые колонки датафрейма:
    - если между буквами стоит знак препинания (кроме "/"), заменяем его на пробел
    - убираем остальные знаки препинания
    - приводим к нижнему регистру
    - убираем лишние пробелы

    Параметры:
        df (pd.DataFrame): исходный датафрейм
        columns (list): список колонок для очистки

    Возвращает:
        pd.DataFrame: обновлённый датафрейм
    """
    def clean(text):

        # 1. Заменяем все знаки препинания между буквами на пробел, кроме "/"
        text = re.sub(r'(?<=[а-яА-Я])([^\w\s\/])(?=[а-яА-Я])', ' ', text)

        # 2. Приводим к нижнему регистру
        text = text.lower()

        # 3. Убираем лишние пробелы
        text = re.sub(r'\s+', ' ', text).strip()

        # 4. Удаляем оставшиеся символы, кроме букв и пробелов
        text = re.sub(r'[^\w\s]', '', text)

        return text

    for col in columns:
        df[col] = df[col].apply(clean)

    return df

In [15]:
# --- Функция ищет отзывы с фразой  key_phrases и с учётом допустимых опечаток ---
def is_fuzzy_match_with_details(text, key_phrases):
    """
    Проверяет, есть ли в тексте фраза, близкая к любой из key_phrases.

    Для слов длиной:
        - 1–3 буквы: точное совпадение или разрешённые замены (e/ё, не/ни)
        - 4–5 букв: расстояние Левенштейна ≤1
        - >5 букв: расстояние Левенштейна ≤2

    Возвращает:
        bool: найдено ли совпадение
        list: список кортежей (original_phrase, corrected_phrase, source_phrase)
    """
    matches = []

    for phrase in key_phrases:
        phrase_words = phrase.split()  # фраза уже разделена на слова
        phrase_len = len(phrase_words)

        # Перебираем возможные позиции начала фразы в отзыве
        for start_idx in range(len(text) - phrase_len + 1):
            match = True
            original_words = []
            corrected_words = []

            for i, target_word in enumerate(phrase_words):
                word = text[start_idx + i]
                original_words.append(word)

                # Если совпадает точно — всё ок
                if word == target_word:
                    corrected_words.append(word)
                    continue

                # Для слов ≤3 букв: только "е" <-> "ё", "не" <-> "ни"
                if len(target_word) <= 3:
                    # Замена "е" <-> "ё"
                    if word.replace("ё", "е") == target_word.replace("ё", "е"):
                        corrected_words.append(word)
                    # Случай "не" <-> "ни"
                    elif target_word == "не" and word == "ни":
                        corrected_words.append(word)
                    elif target_word == "ни" and word == "не":
                        corrected_words.append(word)
                    else:
                        match = False
                        break
                # Для слов длиной 4–5 букв → Левенштейн до 1 ошибки
                elif 4 <= len(target_word) <= 5:
                    distance = lev.distance(word, target_word)
                    if distance <= 1:
                        corrected_words.append(word)
                    else:
                        match = False
                        break
                # Для слов >5 букв → Левенштейн до 2 ошибок
                else:
                    distance = lev.distance(word, target_word)
                    if distance <= 2:
                        corrected_words.append(word)
                    else:
                        match = False
                        break

            # Если фраза подошла и не совпадает точно — добавляем в лог
            if match and ' '.join(corrected_words) != ' '.join(phrase_words):
                matches.append((
                    ' '.join(original_words),
                    ' '.join(corrected_words),
                    phrase
                ))

    return len(matches) > 0, matches

In [16]:
# --- Функция ищет отзывы с точным соответствием фразе key_phrases и с учётом допустимых опечаток ---
import Levenshtein as lev

def is_fuzzy_match_phrase_level(review_words, key_phrases):
    """
    Проверяет, что отзыв полностью соответствует фразе, близкая к любой из key_phrases.

    Для слов длиной:
        - 1–3 буквы: точное совпадение или разрешённые замены (e/ё, не/ни)
        - 4–5 букв: расстояние Левенштейна ≤1
        - >5 букв: расстояние Левенштейна ≤2


    Вход:
        review_words (list): список слов из отзыва (уже очищенный)
        key_phrases (list): список ключевых фраз (уже очищенных и разбитых на слова)

    Выход:
        bool: найдено ли совпадение
        list: список кортежей (original_phrase, corrected_phrase, source_phrase)
    """
    matches = []

    for phrase in key_phrases:
        phrase_words = phrase.split()  # Предполагается, что фраза уже нормализована и разделена на слова

        if abs(len(review_words) - len(phrase_words)) > 1:
            continue

        match = True
        original_words = []
        corrected_words = []

        for i, target_word in enumerate(phrase_words):
            if i >= len(review_words):
                match = False
                break

            word = review_words[i]
            original_words.append(word)

            # Для слов длиной ≤3 букв: только "е" <-> "ё", "не" <-> "ни"
            if len(target_word) <= 3:
                if word == target_word:
                    corrected_words.append(word)
                elif word.replace("ё", "е") == target_word.replace("ё", "е"):
                    corrected_words.append(word)
                elif target_word == "не" and word == "ни":
                    corrected_words.append(word)
                elif target_word == "ни" and word == "не":
                    corrected_words.append(word)
                else:
                    match = False
                    break

            # Слова 4–5 букв → Левенштейн ≤1
            elif 4 <= len(target_word) <= 5:
                distance = lev.distance(word, target_word)
                if distance <= 1:
                    corrected_words.append(word)
                else:
                    match = False
                    break

            # Слова >5 букв → Левенштейн ≤2
            else:
                distance = lev.distance(word, target_word)
                if distance <= 2:
                    corrected_words.append(word)
                else:
                    match = False
                    break

        # Если фраза подошла и не совпадает точно — сохраняем как совпадение
        corrected_phrase = ' '.join(corrected_words)
        phrase_joined = ' '.join(phrase_words)
        if match and corrected_phrase != phrase_joined:
            matches.append((
                ' '.join(original_words),
                corrected_phrase,
                ' '.join(phrase_words)
            ))

    return len(matches) > 0, matches

# **Создание столбца "Класс" и задание для всех отзывов класса "0"**

Создаём объединённые колонки "Отзыв" и "Товар+Отзыв"

In [17]:
# --- Безопасная замена всех некорректных значений на пустую строку ---
for col in ['Текст отзыва', 'Плюсы', 'Минусы']:
    df[col] = df[col].apply(lambda x: '' if not isinstance(x, str) or pd.isna(x) else x)

# Объединяем колонки 'Заголовок отзыва', 'Текст отзыва', 'Плюсы', 'Минусы'
df['Товар+Отзыв'] = df.apply(
    lambda row: f"товар {row['Заголовок отзыва']} {' '.join(filter(None, [row['Текст отзыва'], row['Плюсы'], row['Минусы']]))}",
    axis=1
)

# Объединённый отзыв без сведений о наименовании товара
df['Отзыв']=df['Текст отзыва'].fillna('')+" "+df['Плюсы'].fillna('')+ " "+df['Минусы'].fillna('')

# Заполнение пустых значений в колонке "Продавец"
df['Продавец'] = df['Продавец'].fillna('неизвестный')

# Удаление старых колонок
df.drop(columns=['Текст отзыва', 'Плюсы', 'Минусы'], inplace=True)

# Проверка результата
print("\nДанные после объединения:")
print(df.info())
print()
print('==========================')
print('Пропущенные значения:')
print(df.isnull().sum())  # Проверка наличия пропущенных значений


Данные после объединения:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7136 entries, 0 to 7135
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Оценка             7136 non-null   int64 
 1   Дата создания      7136 non-null   object
 2   Название продукта  7136 non-null   object
 3   Артикул продукта   7136 non-null   object
 4   Продавец           7136 non-null   object
 5   Название бренда    7136 non-null   object
 6   Заголовок отзыва   7136 non-null   object
 7   Товар+Отзыв        7136 non-null   object
 8   Отзыв              7136 non-null   object
dtypes: int64(1), object(8)
memory usage: 501.9+ KB
None

Пропущенные значения:
Оценка               0
Дата создания        0
Название продукта    0
Артикул продукта     0
Продавец             0
Название бренда      0
Заголовок отзыва     0
Товар+Отзыв          0
Отзыв                0
dtype: int64


In [18]:
# Создаём столбец 'Класс' и заполняем его  списком с "0"
df["Класс"] = [[0]] * len(df)

# Создаём столбец 'Класс' с пустой строкой
df["Примечание"] = ''

class_statistics(df)

,Класс,Количество,Процент
0,[0],7136,100.0
1,Всего,7136,100.0


# Группа классов №5

 **Анализ исходных данных**

---



## **Модель №1 s-nlp/ruT5-base-detox**

In [ ]:
import pandas as pd
import time
import sys
from transformers import AutoTokenizer, T5ForConditionalGeneration
import torch

In [ ]:
# --- Загрузка модели и токенизатора ---
base_model_name = 'ai-forever/ruT5-base'
model_name = 's-nlp/ruT5-base-detox'

tokenizer = AutoTokenizer.from_pretrained(base_model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/20.4k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.00M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [ ]:
def contains_profanity(original_text, model, tokenizer, max_words_per_chunk=10):
    """
    Проверяет, содержит ли текст нецензурную лексику,
    разбивая его на чанки по max_words_per_chunk слов.

    Возвращает:
        bool: найдена ли нецензурная лексика
        str: очищенный текст
    """
    if not isinstance(original_text, str) or not original_text.strip():
        return False, original_text

    words = original_text.split()
    total_words = len(words)

    cleaned_chunks = []
    is_profane = False

    for i in range(0, total_words, max_words_per_chunk):
        chunk = ' '.join(words[i:i + max_words_per_chunk])

        input_ids = tokenizer.encode(chunk, return_tensors='pt', truncation=True, max_length=512)
        with torch.no_grad():
            output_ids = model.generate(input_ids, max_new_tokens=50, num_return_sequences=1)
        cleaned_chunk = tokenizer.decode(output_ids[0], skip_special_tokens=True)

        # Приводим оба текста к нижнему регистру для корректного сравнения
        original_chunk_lower = chunk.lower().strip()
        cleaned_chunk_lower = cleaned_chunk.lower().strip()

        if cleaned_chunk_lower != original_chunk_lower:
            is_profane = True

        cleaned_chunks.append(cleaned_chunk)

    # Собираем полный очищенный текст
    cleaned_text = ' '.join(cleaned_chunks)

    # Принудительно переводим весь текст в нижний регистр
    cleaned_text = cleaned_text.lower()

    return is_profane, cleaned_text


# --- Функция проставления кода класса ---
def set_class(df, idx, code):
    """
    Проставляет код класса. Если ранее был 0 — ставим code.
    Если уже стоит другой код — оставляем как есть.
    """
    current_class = df.at[idx, 'Класс']

    if current_class == 0:
        df.at[idx, 'Класс'] = code
    elif current_class != code:
        # Например, если вы хотите хранить несколько меток
        if isinstance(current_class, list):
            df.at[idx, 'Класс'] = current_class + [code]
        else:
            df.at[idx, 'Класс'] = [current_class, code]


# --- Классификатор с обработкой чанков и диапазона строк ---
def classifier_500(
    df,
    code=500,
    log_profanity=False,
    start_row=0,
    end_row=None,
    max_words_per_chunk=10
):
    """
    Присваивает класс code=500, если в отзыве найдена нецензурная лексика.
    Использует модель ruT5-base-detox для анализа.

    Параметры:
        df: исходный DataFrame
        code: код класса для пометки мата
        log_profanity: сохранять ли найденные маты в отдельный список
        start_row: начальная строка для обработки
        end_row: конечная строка для обработки
        max_words_per_chunk: максимальное количество слов в одном чанке
    """

    # Ограничиваем диапазон строк
    if end_row is None:
        end_row = len(df)

    subset_df = df.iloc[start_row:end_row]
    total_rows = len(subset_df)
    processed = 0
    start_time = time.time()

    detected_profanity_reviews = []

    print(f"\n🚀 Начинаем обработку строк с {start_row} по {end_row - 1}")

    for idx, row in subset_df.iterrows():
        original_text = row['Отзыв']

        if not isinstance(original_text, str) or not original_text.strip():
            processed += 1
            continue

        is_profane, cleaned_text = contains_profanity(original_text, model, tokenizer, max_words_per_chunk)

        if is_profane:
            set_class(df, idx, code)

            if log_profanity:
                detected_profanity_reviews.append({
                    'Номер строки': idx,
                    'Оригинальный отзыв': original_text,
                    'После детоксикации': cleaned_text,
                    'Код класса': code
                })

        processed += 1

        # Отображаем прогресс
        percent = (processed / total_rows) * 100
        elapsed = time.time() - start_time
        sys.stdout.write(f"\r🔍 Обработано {processed} из {total_rows} ({percent:.1f}%)")
        sys.stdout.flush()

    print("\n✅ Обработка завершена")

    return df, detected_profanity_reviews



# --- Вызов функции ---

# Добавляем колонку для метки класса, если её нет
if 'Класс' not in df.columns:
    df['Класс'] = 0

# Указываем диапазон строк для обработки
start_row = 0
end_row = 20  # например, обрабатываем первые 100 строк

# Вызываем классификатор
df, profanity_log = classifier_500(
    df,
    code=500,
    log_profanity=True,
    start_row=start_row,
    end_row=end_row,
    max_words_per_chunk=10  # параметр разбиения на чанки
)

# --- Сохранение результата ---
if profanity_log:
    profanity_df = pd.DataFrame(profanity_log)
    profanity_df.to_excel('обнаруженная_нецензура_500.xlsx', index=False)
    print(f"✅ Найдено {len(profanity_log)} отзывов с нецензурной лексикой")
else:
    print("❌ Нецензурных отзывов не найдено.")

# --- Результат ---
print("📌 Первая 10 строк после обработки:")
print(df.iloc[start_row:start_row+10][['Отзыв', 'Класс']])


🚀 Начинаем обработку строк с 0 по 19
🔍 Обработано 20 из 20 (100.0%)
✅ Обработка завершена
✅ Найдено 12 отзывов с нецензурной лексикой
📌 Первая 10 строк после обработки:
                                               Отзыв       Класс
0     обычная только фирма ок включатели не надежные         [0]
1  после 3х месяцев эксплуатации потекла вода пом...         [0]
2  не советую брать ничего колонка не подключаетс...    [0, 500]
3                              прислали другой товар       [102]
4  упаковкой играли футбол что после может быть в...         [0]
5  просьба вернуть деньги за негодный товар вообщ...       [800]
6                             подошва вся покарябана  [102, 500]
7  ребята если хотите испортить настроение себе и...    [0, 500]
8                не работает делали как в инструкции  [800, 500]
9  утюг не проработал и полгода перестал отпарива...         [0]


# **Модель №2 check_swear**

In [ ]:
!pip install check-swear

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 44.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: joblib
    Found existing installation: joblib 1.5.0
    Uninstalling joblib-1.5.0:
      Successfully uninstalled joblib-1.5.0
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the

In [ ]:
from check_swear import SwearingCheck
import pandas as pd

In [ ]:
from check_swear import SwearingCheck
import pandas as pd
import time
import sys

# --- Инициализация модели ---
sch = SwearingCheck()

# --- Функция получения вероятности мата ---
def get_profanity_probability(text):
    try:
        proba = sch.predict_proba([text])
        if isinstance(proba, (list, tuple)):
            return float(proba[0])
        elif hasattr(proba, 'tolist'):
            return float(proba.tolist()[0])
        else:
            return float(proba)
    except Exception as e:
        print(f"Ошибка при обработке текста '{text}': {e}")
        return 0.0

# --- Основная функция: classifier_500_2 ---
def classifier_500_2(
    df,
    code=501,
    log_profanity=True,
    start_row=0,
    end_row=None,
    threshold=0.5
):
    """
    Классификатор мата на основе check-swear.

    Параметры:
        df: DataFrame с колонкой 'Отзыв'
        code: код класса, который проставляется, если найден мат
        log_profanity: сохранять ли логи в отдельный список
        start_row: начальная строка для обработки
        end_row: конечная строка для обработки
        threshold: порог вероятности (например, 0.5)
    """
    # Ограничиваем диапазон строк
    if end_row is None:
        end_row = len(df)

    subset_df = df.iloc[start_row:end_row]
    total_rows = len(subset_df)
    processed = 0
    start_time = time.time()

    # Логгер
    detected_profanity_reviews = []

    print(f"\n🚀 Начинаем обработку строк с {start_row} по {end_row - 1}")

    for idx, row in subset_df.iterrows():
        original_text = row['Отзыв']

        if not isinstance(original_text, str) or not original_text.strip():
            processed += 1
            continue

        # Получаем вероятность мата
        probability = get_profanity_probability(original_text)
        is_profane = probability > threshold

        if is_profane:
            set_class(df, idx, code)

            if log_profanity:
                detected_profanity_reviews.append({
                    'Номер строки': idx,
                    'Отзыв': original_text,
                    'Вероятность мата': round(probability, 4),
                    'Код класса': code
                })

        processed += 1

        # Обновление прогресса
        percent = (processed / total_rows) * 100
        elapsed = time.time() - start_time
        sys.stdout.write(f"\r🔍 Обработано {processed} из {total_rows} ({percent:.1f}%)")
        sys.stdout.flush()

    print("\n✅ Обработка завершена")

    return df, detected_profanity_reviews


# --- Пример использования ---


# --- Вызов функции ---
start_row = 0
end_row = len(df)  # можно указать, например, 100 для первых 100 строк
threshold = 0.5  # порог вероятности

df, profanity_log = classifier_500_2(
    df,
    code=501,
    log_profanity=True,
    start_row=start_row,
    end_row=end_row,
    threshold=threshold
)

# --- Сохранение результата ---
print("\n📌 Результаты:")
print(df[['Отзыв', 'Класс']])

if profanity_log:
    profanity_df = pd.DataFrame(profanity_log)
    profanity_df.to_excel('обнаруженная_нецензура_501.xlsx', index=False)
    print(f"\n✅ Найдено {len(profanity_log)} отзывов с нецензурной лексикой")
else:
    print("\n❌ Нецензурных отзывов не найдено.")


🚀 Начинаем обработку строк с 0 по 7135
🔍 Обработано 7132 из 7136 (99.9%)
✅ Обработка завершена

📌 Результаты:
                                                  Отзыв     Класс
0        Обычная,только фирма ок Включатели не надежные       [0]
1                                                             [0]
2     После 3х месяцев эксплуатации потекла вода пом...       [0]
3     Не советую брать Ничего Колонка не подключаетс...  [0, 501]
4                                 Прислали другой товар       [0]
...                                                 ...       ...
7131  Тяжёлый очень. Одной рукой держать на весу оч....  [0, 501]
7132                                                          [0]
7133                                                          [0]
7134                                                          [0]
7135                                                          [0]

[7136 rows x 2 columns]

✅ Найдено 14 отзывов с нецензурной лексикой


# **Модель №4 SkolkovoInstitute/russian_toxicity_classifier**

In [ ]:
!pip install transformers torch pandas

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import pandas as pd
import time
import sys

In [ ]:
# --- Шаг 3: Загрузка модели и токенизатора ---
model_name = "SkolkovoInstitute/russian_toxicity_classifier"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [ ]:
# --- Шаг 4: Функция определения токсичности ---
def is_toxic(text, threshold=0.5):
    if not isinstance(text, str) or not text.strip():
        return False, 0.0

    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        logits = model(**inputs).logits
    probs = torch.softmax(logits, dim=1).numpy()
    toxic_prob = probs[0][1]
    return toxic_prob > threshold, float(toxic_prob)

# --- Шаг 5: Ваша функция set_class() ---
def set_class(df, idx, code):
    """
    Присваивает класс с указанным кодом.
    Если [0] → заменить на [code]
    Если нет 0 и нет code → добавляем code
    Иначе → ничего не делаем
    """
    current_class = df.at[idx, 'Класс']

    if current_class == [0]:
        df.at[idx, 'Класс'] = [code]
    elif 0 not in current_class and code not in current_class:
        df.at[idx, 'Класс'] = current_class + [code]

    return df

# --- Шаг 6: Новая функция classifier_500() ---
def classifier_500(df, text_col='Отзыв', code=500, threshold=0.7, start_row=0, end_row=None):
    """
    Проверяет тексты на токсичность и проставляет класс code, если найден мат.

    Параметры:
        df: DataFrame с колонкой текста
        text_col: имя колонки с отзывами
        code: код класса для пометки мата
        threshold: порог вероятности токсичности
        start_row: начальная строка
        end_row: конечная строка
    """

    if end_row is None:
        end_row = len(df)

    subset_df = df.iloc[start_row:end_row]
    total = len(subset_df)
    start_time = time.time()
    detected = []

    print(f"\n🚀 Начинаем обработку строк с {start_row} по {end_row - 1}")

    for idx, row in subset_df.iterrows():
        text = row[text_col]

        try:
            is_toxic_flag, score = is_toxic(text, threshold)
        except Exception as e:
            print(f"\n❌ Ошибка при обработке строки {idx}: {e}")
            continue

        if is_toxic_flag:
            df = set_class(df, idx, code)
            detected.append({
                'Номер строки': idx,
                'Текст': text,
                'Вероятность токсичности': round(score, 4),
                'Код класса': code
            })

        # Вывод прогресса
        percent = ((idx - start_row + 1) / total) * 100
        sys.stdout.write(f"\r🔍 Обработано {idx - start_row + 1} из {total} ({percent:.1f}%)")
        sys.stdout.flush()

    print("\n✅ Обработка завершена")

    return df, detected

# Классификация

# --- Шаг 8: Вызов функции ---
start_row = 0
end_row = len(df)
threshold = 0.9

df, logs = classifier_500(
    df,
    text_col='Отзыв',
    code=500,
    threshold=threshold,
    start_row=start_row,
    end_row=end_row
)

# --- Шаг 9: Сохранение результата ---
print("\n📌 Результаты:")
print(df[['Отзыв', 'Класс']])

if logs:
    log_df = pd.DataFrame(logs)
    log_df.to_excel('обнаруж_нецензура_модель №3_0.9.xlsx', index=False)
    print(f"\n✅ Найдено {len(logs)} отзывов с нецензурной лексикой")
else:
    print("\n❌ Нецензурных отзывов не найдено.")


🚀 Начинаем обработку строк с 0 по 7135
🔍 Обработано 7136 из 7136 (100.0%)
✅ Обработка завершена

📌 Результаты:
                                                  Отзыв     Класс
0        Обычная,только фирма ок Включатели не надежные       [0]
1                                                             [0]
2     После 3х месяцев эксплуатации потекла вода пом...       [0]
3     Не советую брать Ничего Колонка не подключаетс...  [0, 501]
4                                 Прислали другой товар       [0]
...                                                 ...       ...
7131  Тяжёлый очень. Одной рукой держать на весу оч....  [0, 501]
7132                                                          [0]
7133                                                          [0]
7134                                                          [0]
7135                                                          [0]

[7136 rows x 2 columns]

✅ Найдено 95 отзывов с нецензурной лексикой


# **Модель №5 cointegrated/rubert-tiny-toxicity**

In [ ]:
!pip install transformers torch pandas tqdm emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 765.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 35.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninst

In [ ]:
# --- Шаг 2: Импорт ---
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import pandas as pd
import re
import time
import sys

# --- Шаг 3: Загрузка модели и токенизатора ---
model_name = "cointegrated/rubert-tiny-toxicity"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Если доступен GPU — используем его
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
print(f"🧠 Модель загружена. Используется: {device.upper()}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


🧠 Модель загружена. Используется: CPU


In [ ]:
# --- 1. Ваш основной словарь мата ---
BAD_WORDS = {
    'хуй', 'пизда', 'блядь', 'ебать', 'мудак', 'сука', 'говно',
    'нахуй', 'пшел', 'идиот', 'придурок', 'даун', 'лох',
}

# --- 2. Слова-исключения ---
EXCEPTION_WORDS = {
    'мудрый', 'мудрец', 'мудрость',
    'пистолет', 'пистон', 'пистолетик',
    'бляха', 'бляха-муха', 'бляшки',
    'ефрейтор', 'ефимия', 'ефир',
    'говяжий', 'говядина', 'говяжка',
    'ходил', 'ходит', 'ходы', 'ходяга',
    'утиная', 'утилитаризм', 'утюг', 'утка', 'утки', 'утюжить',
    'иди', 'идите', 'идёт', 'идем', 'идиотка', 'идиотизм',
    'тупик', 'тупая', 'тупой', 'тупиков', 'тупо', 'тупить',
    'сукно', 'суконный', 'суконная юбка', 'суконный рынок',
    'суконный дом', 'суконный магазин', 'сукно на платье',
    'сучья', 'сучкорез',
    'говяжья котлета', 'говяжий бульон', 'говяжий фарш',
    'пизанская башня', 'пизанское вертикальное отклонение',
    'пизанский собор', 'пизанские учёные'
}

# --- 3. Регулярное выражение с учетом исключений ---
def build_profanity_pattern(bad_words, exceptions):
    """
    Создает регулярное выражение, которое:
    - Находит слова из BAD_WORDS
    - Исключает слова из EXCEPTION_WORDS
    """
    # Экранируем слова для безопасного использования в regex
    bad_pattern = r'\b(?:{})\b'.format('|'.join(map(re.escape, bad_words)))

    # Создаем шаблон исключений
    exception_pattern = r'(?:' + '|'.join(map(re.escape, exceptions)) + r')'

    # Финальный паттерн: находим мат, но исключаем совпадения с EXCEPTION_WORDS
    full_pattern = fr'{bad_pattern}(?!\w)(?!.*(?:{exception_pattern}))'

    return re.compile(full_pattern, re.IGNORECASE)

# --- 4. Компилируем регулярку ---
profanity_pattern = build_profanity_pattern(BAD_WORDS, EXCEPTION_WORDS)

def contains_profanity(text):
    """
    Проверяет наличие мата, игнорируя слова-исключения
    """
    if not isinstance(text, str) or not text.strip():
        return False

    return bool(profanity_pattern.search(text))


# --- Шаг 6: Функция проверки токсичности отзыва через чанки ---
def is_toxic_with_chunks(text, threshold=0.6, chunk_size=50):
    if not isinstance(text, str) or not text.strip():
        return False, 0.0

    # Сначала проверяем через свой словарь (быстро и точно)
    if contains_profanity(text):
        return True, 0.99  # принудительно ставим высокую вероятность

    # Делим текст на чанки
    chunks = split_text_into_chunks(text, chunk_size)

    toxic_scores = []
    for chunk in chunks:
        inputs = tokenizer(chunk, return_tensors="pt", truncation=True, padding=True).to(device)
        with torch.no_grad():
            logits = model(**inputs).logits
        probs = torch.softmax(logits, dim=1).cpu().numpy()
        toxic_prob = probs[0][1]
        toxic_scores.append(toxic_prob)

    max_score = max(toxic_scores) if toxic_scores else 0.0
    return max_score > threshold, max_score

In [ ]:
# --- Шаг 7: Функция set_class (ваша) ---
def set_class(df, idx, code):
    current_class = df.at[idx, 'Класс']

    if current_class == [0]:
        df.at[idx, 'Класс'] = [code]
    elif 0 not in current_class and code not in current_class:
        df.at[idx, 'Класс'] = current_class + [code]

    return df

# --- Шаг 8: Классификатор с чанками и правилами ---
def classifier_500(df, text_col='Отзыв', code=500, threshold=0.6, start_row=0, end_row=None, chunk_size=50):
    """
    Проверяет отзывы на токсичность, разбивая длинные тексты на чанки
    """

    if end_row is None:
        end_row = len(df)

    subset_df = df.iloc[start_row:end_row]
    total = len(subset_df)
    detected = []

    print(f"\n🚀 Начинаем обработку строк с {start_row} по {end_row - 1}")

    for idx, row in subset_df.iterrows():
        text = row[text_col]

        try:
            flag, score = is_toxic_with_chunks(text, threshold, chunk_size)
        except Exception as e:
            print(f"\n❌ Ошибка при обработке строки {idx}: {e}")
            continue

        if flag:
            df = set_class(df, idx, code)
            detected.append({
                'Номер строки': idx,
                'Текст': text,
                'Вероятность токсичности': round(score, 4),
                'Код класса': code
            })

        # Отображение прогресса
        percent = ((idx - start_row + 1) / total) * 100
        sys.stdout.write(f"\r🔍 Обработано {idx - start_row + 1} из {total} ({percent:.1f}%)")
        sys.stdout.flush()

    print("\n✅ Обработка завершена")

    return df, detected



# --- Шаг 10: Вызов функции ---
start_row = 0
end_row = len(df)
threshold = 0.4
chunk_size = 50

df, logs = classifier_500(
    df,
    text_col='Отзыв',
    code=500,
    threshold=threshold,
    start_row=start_row,
    end_row=end_row,
    chunk_size=chunk_size
)

# --- Шаг 11: Сохранение результата ---
print("\n📌 Результаты:")
print(df[['Отзыв', 'Класс']])

if logs:
    log_df = pd.DataFrame(logs)
    log_df.to_excel('обнаруж_нецензура_модель №5_1_0,4.xlsx', index=False)
    print(f"\n✅ Найдено {len(logs)} отзывов с нецензурной лексикой")
else:
    print("\n❌ Нецензурных отзывов не найдено.")


🚀 Начинаем обработку строк с 0 по 7135
🔍 Обработано 7136 из 7136 (100.0%)
✅ Обработка завершена

📌 Результаты:
                                                  Отзыв Класс
0        Обычная,только фирма ок Включатели не надежные   [0]
1                                                         [0]
2     После 3х месяцев эксплуатации потекла вода пом...   [0]
3     Не советую брать Ничего Колонка не подключаетс...   [0]
4                                 Прислали другой товар   [0]
...                                                 ...   ...
7131  Тяжёлый очень. Одной рукой держать на весу оч....   [0]
7132                                                      [0]
7133                                                      [0]
7134                                                      [0]
7135                                                      [0]

[7136 rows x 2 columns]

✅ Найдено 19 отзывов с нецензурной лексикой


# **Композиция моделей**

In [1]:
!pip install transformers torch pandas tqdm check-swear

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 37.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: joblib
    Found existing installation: joblib 1.5.1
    Uninstalling joblib-1.5.1:
      Successfully uninstalled joblib-1.5.1
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the

In [1]:

# --- Шаг 2: Импорт ---
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from check_swear import SwearingCheck
import pandas as pd
import re
import time
import sys

# --- Шаг 3: Загрузка rubert-tiny-toxicity ---
model_name_rubert = "cointegrated/rubert-tiny-toxicity"
tokenizer_rubert = AutoTokenizer.from_pretrained(model_name_rubert)
model_rubert = AutoModelForSequenceClassification.from_pretrained(model_name_rubert)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_rubert.to(device)

# --- Шаг 4: Загрузка check_swear ---
swearing_checker = SwearingCheck()

tokenizer_config.json:   0%|          | 0.00/377 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/47.2M [00:00<?, ?B/s]

In [36]:
# --- Шаг 5: Ваш список мата и исключений ---
BAD_WORDS = {
    'х**', 'х**во', 'п**да', 'пи-ды', 'п**дец', 'мудак', 'сука', 'с*ка', 'с*ки',
    'с-ка', 'с-ки','говно',
    'пшел', 'придурок', 'даун', 'дерьмо', 'урод',
    'хрень', 'херовый', 'тварь'
}

EXCEPTION_WORDS = {
    'сучья', 'сучкорез',
}

# --- Шаг 6: Функция для поиска мата с поддержкой частичных совпадений ---
def build_profanity_pattern(bad_words):
    # Теперь ищем любое вхождение BAD_WORDS в тексте (без границ \b)
    bad_pattern = r'(?:{})'.format('|'.join(map(re.escape, bad_words)))
    return re.compile(bad_pattern, re.IGNORECASE)

# --- Шаг 7: Функция проверки исключений ---
def build_exception_pattern(exceptions):
    exception_pattern = r'(?:{})'.format('|'.join(map(re.escape, exceptions)))
    return re.compile(exception_pattern, re.IGNORECASE)

# --- Шаг 8: Компилируем регулярки ---
profanity_regex = build_profanity_pattern(BAD_WORDS)
exception_regex = build_exception_pattern(EXCEPTION_WORDS)

def contains_profanity(text):
    if not isinstance(text, str) or not text.strip():
        return False

    # Проверяем, есть ли мат и нет ли исключений
    has_profanity = bool(profanity_regex.search(text))
    has_exception = bool(exception_regex.search(text))

    return has_profanity and not has_exception

# --- Шаг 7: Функция проверки rubert-tiny-toxicity ---
RUBERT_THRESHOLD = 0.7
def get_rubert_toxicity(text, chunk_size=50):
    if not isinstance(text, str) or not text.strip():
        return 0.0

    # Сначала проверяем через ваш словарь мата
    if contains_profanity(text):
        return 0.99  # явный мат из словаря → высокая уверенность

    # Если нет совпадений, отправляем в модель
    clean_text = re.sub(r'[^\w\s]', '', text.lower())
    words = clean_text.split()
    scores = []

    for i in range(0, len(words), chunk_size):
        chunk = ' '.join(words[i:i + chunk_size])
        inputs = tokenizer_rubert(chunk, return_tensors="pt", truncation=True, padding=True).to(device)

        with torch.no_grad():
            logits = model_rubert(**inputs).logits

        probs = torch.softmax(logits, dim=1).cpu().numpy()
        scores.append(probs[0][1])

    max_score = max(scores) if scores else 0.0
    return round(float(max_score), 4)

# --- Шаг 8: Функция проверки check_swear ---
CHECKSWEAR_THRESHOLD = 0.5
def get_check_swear_score(text):
    if not isinstance(text, str) or not text.strip():
        return 0.0

    try:
        score = swearing_checker.predict_proba([text])[0]
        if isinstance(score, (list, tuple)):
            score = score[0]
        return round(float(score), 4)
    except Exception as e:
        print(f"⚠️ Ошибка в check_swear: {e}")
        return 0.0

# --- Шаг 9: Обработка датафрейма с двумя моделями и порогами ---
def process_reviews(df, text_col='Отзыв', level_1=0.6):
    results = []
    total = len(df)
    start_time = time.time()

    for idx, row in df.iterrows():
        text = row[text_col]

        # Получаем вероятности от каждой модели
        proba_rubert = get_rubert_toxicity(text)
        proba_check_swear = get_check_swear_score(text)

        # Суммарная вероятность (можно использовать max или среднее)
        combined_score = max(proba_rubert, proba_check_swear)
        final_flag = combined_score > level_1

        # Добавляем в результаты
        results.append({
            'Текст отзыва': text,
            'rubert_tiny': proba_rubert,
            'check_swear': proba_check_swear,
            'sum_probanility': round(combined_score, 4),
            'class_500': 500 if final_flag else 0
        })

        # Прогрессбар
        percent = (idx + 1) / total * 100
        sys.stdout.write(f"\r🔍 Обработано {idx + 1} из {total} ({percent:.1f}%)")
        sys.stdout.flush()

    print("\n✅ Обработка завершена")
    result_df = pd.DataFrame(results)
    return result_df


# --- Шаг 11: Вызов функции обработки ---
LEVEL_1 = 0.65 # порог для суммарной оценки

result_df = process_reviews(df, text_col='Отзыв', level_1=LEVEL_1)

# --- Шаг 12: Сохранение результата ---
print("\n📌 Результаты:")
print(result_df[['Текст отзыва', 'sum_probanility', 'class_500']])

result_df.to_excel('результат_две_модели_с_классом.xlsx', index=False)
print("\n💾 Результаты сохранены в файл 'результат_две_модели_с_классом.xlsx'")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


🔍 Обработано 7136 из 7136 (100.0%)
✅ Обработка завершена

📌 Результаты:
                                           Текст отзыва  sum_probanility  \
0        обычная только фирма ок включатели не надежные           0.0000   
1                                                                 0.0000   
2     после 3х месяцев эксплуатации потекла вода пом...           0.0000   
3     не советую брать ничего колонка не подключаетс...           0.3333   
4                                 прислали другой товар           0.0002   
...                                                 ...              ...   
7131  тяжёлый очень одной рукой держать на весу оч т...           0.3333   
7132                                                              0.0000   
7133                                                              0.0000   
7134                                                              0.0000   
7135                                                              0.0000   

      class_500